In [ ]:
import pandas as pd
import numpy as np
import scipy.sparse as sp

Импортируем нужные библиотеки

In [ ]:
def get_sim(item_id):
    rat = [(other_item_id, np.linalg.norm(H[:, item_id - 1] - H[:, other_item_id - 1]))
           for other_item_id in ratings["movie_id"].unique()]
    items_ratings = sorted(rat, key=lambda x: x[1])
    return items_ratings[:10]


def get_recom(user_id):
    unique_movies = set(movies.to_numpy())
    for k in range(len(ratings_np)):
        if ratings_np[k][0] == user_id:
            unique_movies.remove(ratings_np[k][1])
    unique_movies_arr = []
    for t in unique_movies:
        unique_movies_arr.append(t)
    new_items_ratings = (W @ H)[user_id - 1][np.array(unique_movies_arr) - 1]
    items_ratings = sorted(list(zip(unique_movies, new_items_ratings)), key=lambda x: x[1], reverse=True)
    return items_ratings[:10]

def get_best_neg(user, pos_V, pos_items):
    neg_items = set()
    while len(neg_items) < sample_size:
        neg_item = np.random.randint(items_size)
        if pos_items.__contains__(neg_item + 1):
            continue
        else:
            neg_items.add(neg_item)
    cur_V = 1e6
    ans_V = 0
    ans_item = 0
    for neg_item in neg_items:
        neg_V = W[user, :] @ H[:, neg_item]
        if abs(neg_V - pos_V) < cur_V:
            cur_V = abs(neg_V - pos_V)
            ans_V = neg_V
            ans_item = neg_item
    return ans_V, ans_item

Три дополнительных метода, чтобы считать симилары для конкретного айтема, рекомендации для конкретного юзера и "наилучший" негативный айтем

In [ ]:
ratings = pd.read_csv('ratings.dat', delimiter='::', header=None,
                      names=['user_id', 'movie_id', 'rating', 'timestamp'],
                      usecols=['user_id', 'movie_id', 'rating'], engine='python')

movie_info = pd.read_csv('movies.dat', delimiter='::', header=None,
                         names=['movie_id', 'name', 'category'], engine='python')

users = ratings["user_id"]
movies = ratings["movie_id"]
ratings_np = ratings.to_numpy()
users_size = users.max()
items_size = movies.max()
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]
implicit_ratings_np = implicit_ratings.to_numpy()
user_item = sp.coo_matrix(
    (np.ones_like(implicit_ratings["user_id"]), (implicit_ratings["user_id"], implicit_ratings["movie_id"])))
user_item_csr_arr = user_item.tocsr().toarray()

Подгружаем наш датасет

In [ ]:
K = 64
eps = 1e-3
lam = 1e-5
sample_size = 100
count_iter = 100

Подбираем гиперпараметры модели

In [ ]:
W = np.random.uniform(0, 1 / np.sqrt(K), size=(users_size, K))
H = np.random.uniform(0, 1 / np.sqrt(K), size=(K, items_size))

Инициализируем матрицы юзеров (W) и айтемов (H)

In [ ]:
for t in range(10):
    print("эпоха {}".format(t))
    for user in range(users_size):
        pos_items = set(user_item_csr_arr[user + 1, 1:].nonzero()[0])
        for pos_item in pos_items:
            pos_item -= 1
            pos_V = W[user, :] @ H[:, pos_item]
            neg_V, neg_item = get_best_neg(user, pos_V, pos_items)
            e = 1 / (1 + np.exp(pos_V - neg_V))
            W[user, :] += eps * (e * (H[:, pos_item] - H[:, neg_item]).T - lam * W[user, :])
            H[:, pos_item] += eps * (e * W[user, :].T - lam * H[:, pos_item])
            H[:, neg_item] += eps * (e * W[user, :].T - lam * H[:, neg_item])

Обучаем нашу модель на implicit данных

In [ ]:
get_similars = lambda item_id, model: [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() for x in model]
print(get_similars(1, get_sim(1)))

In [ ]:
get_recommendations = lambda user_id, model: [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() for x in model]
print(get_recommendations(4, get_recom(4)))